# MCMC Code
## School Research Project
### Written by Yolanda Dube

In [ ]:
# Importing the necessary packages

import emcee
import latex
import numpy as np
from classy import Class
import multiprocessing as mp
from functools import partial
import getdist.plots as gdplots
import matplotlib.pyplot as plt
import matplotlib.font_manager
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
from multiprocessing import Pool
from scipy.integrate import quad
import matplotlib.ticker as ticker
from getdist import plots, MCSamples
import matplotlib.patches as mpatches
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LogLocator
from scipy.interpolate import CubicSpline
from matplotlib.ticker import LogFormatter
from matplotlib.pyplot import rc, rcParams
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import FormatStrFormatter

In [ ]:
k_values = np.linspace(0.005, 0.030, 100)
z_value = 0.75

In [ ]:
# Defining the Cosmological Parameters

cosmo = Class()
params = {
    'output'      : 'mPk',                # Request matter power spectrum calculation
    'omega_b'     : 0.0223828,            # Baryon density
    'omega_cdm'   : 0.1201075,            # Cold Dark Matter density
    'h'           : 0.6736,               # Hubble parameter
    'A_s'         : 2.1e-9,               # Amplitude of primordial power spectrum
    'n_s'         : 0.9665,               # Spectral index of primordial power spectrum
    'tau_reio'    : 0.0543,               # Optical depth to reionization
    'Omega_Lambda': 0.6825,               # Present-day density parameter for dark energy
    'Omega_k'     : 0.0,                  # Present-day density parameter for curvature
    'z_max_pk'    : z_value,                 # Maximum redshift for storing power spectrum
}

cosmo.set(params)

In [ ]:
# Defining the Survey Parameters

survey_params = [
    {
        "name"    : "MeerKLASS L-band",
        "V_survey": 1.3e9,
        "f_sky"   : 0.10,
        "nu_min"  : 900.0,
        "nu_max"  : 1185.0,
        "t_obs"   : 4000.0,
        "N_dish"  : 64,
        "k_min"   : 3.3
    },
    {
        "name"    : "MeerKLASS UHF-band",
        "V_survey": 10.8e9,
        "f_sky"   : 0.10,
        "nu_min"  : 580.0,
        "nu_max"  : 1000.0,
        "t_obs"   : 4000.0,
        "N_dish"  : 64,
        "k_min"   : 1.6
    },
    {
        "name"    : "SKA-MID Band 1",
        "V_survey": 221.6e9,
        "f_sky"   : 0.48,
        "nu_min"  : 350.0,
        "nu_max"  : 1050.0,
        "t_obs"   : 20000.0,
        "N_dish"  : 197,
        "k_min"   : 0.53
    },
]

In [ ]:
# The HI-LIne Brightness Temperature

def T_HI(z):
    cosmo.compute()
    Omega_HI = 0.00048 + 0.00039*z - 0.000065*z**2
    h_Hubble = cosmo.h()
    H_z = cosmo.Hubble(z)/2997.92
    H_0 = cosmo.Hubble(0)/2997.92
    T_HI = ((180*Omega_HI*h_Hubble*(1+z)**2)/(H_z/H_0))*(1e-3) # Converting to Kelvins
    return T_HI

In [ ]:
# The Linear Bias

def b_HI(z):
    b_HI = 0.842 + 0.693*z - 0.046*z**2
    return b_HI

In [ ]:
# The comoving distance

def chi(z):
        cosmo.compute()
        chi, _ = quad(lambda x: 1 / (cosmo.Hubble(x)/2997.92) , 0, z)
        return chi

In [ ]:
# Lambda(z) 

def lambda_z(z):
        return (0.21/3.0857e22) * (1 + z)

In [ ]:
# The growth rate f(z) 

def growth_rate(z):
    cosmo.compute()
    a = 1.0 / (1.0 + z)
    H = cosmo.Hubble(z)/2997.92  # Hubble parameter at redshift z
    # We Compute the growth factor D at redshift z
    D = cosmo.scale_independent_growth_factor(z)
    # Compute the derivative of D with respect to the scale factor a
    # This can be done numerically using a small delta_a
    delta_a = 1e-5
    D_plus = cosmo.scale_independent_growth_factor(1.0 / (a + delta_a) - 1.0)
    dD_da = (D_plus - D) / delta_a
    # We Calculate the growth rate f Using
    f = (a / D) * dD_da
    return f

In [ ]:
# The HI Power Spectrum : Not marginalized over mu

def P_HI(k, mu, z):
    cosmo.compute()
    f = cosmo.scale_independent_growth_factor(z)
    P_HI =  T_HI(z)**2 *(b_HI(z) + growth_rate(z)*mu**2)**2 * cosmo.pk(k,z)
    return P_HI

In [ ]:
# The Beam Factor 

def D_b(k, mu, z):
    #D_dish = survey["D_dish"]
    cosmo.compute()
    chi, _ = quad(lambda x: 1 / cosmo.Hubble(x), 0, z)
    Theta_b = lambda z: 1.22 * (0.21/3.085e22) * (1 + z) / 13.5
    D_b = np.exp(-((1 - mu ** 2) * k ** 2 * chi ** 2 * Theta_b(z) ** 2) / (16 * np.log(2)))
    return D_b

In [ ]:
# The Foreground Factor

def D_fg(k, mu, z, k_para_fg):
    D_fg = 1 - np.exp(-(mu * k / k_para_fg) ** 2)
    return D_fg

In [ ]:
# The marginalized power spectrum 

def P_HI_kz(k,z):
    P_HI_kz_values = np.zeros_like(k)
    for i, k_value in enumerate(k):
        P_HI_kz_value, _ = quad(lambda mu: P_HI(k_value, mu, z)*D_fg(k_value, mu, z, 0.001)*D_b(k_value, mu, z), -1, 1)
        P_HI_kz_values[i] = P_HI_kz_value
    return P_HI_kz_values

In [ ]:
# Plotting the marginalized power spectrum 

y_values = P_HI_kz(k_values, z_value)
plt.loglog(k_values, y_values)
plt.show()

In [ ]:
# Thermal noise expression

def calculate_noise_power(survey, k, z):
    nu_min = survey["nu_min"]
    nu_max = survey["nu_max"]
    t_obs = survey["t_obs"]
    N_dish = survey["N_dish"]
    f_sky = survey["f_sky"]

    T_gal =lambda nu: 25.0 * (408.0 / nu) ** 2.75

    T_rx = lambda nu: 7.5 + 10.0 * ((nu*1e3) - 0.75) ** 2
        
    T_gal_total, _ = quad(lambda nu: T_gal(nu) + T_rx(nu), nu_min, nu_max)    

    T_sys = (3.0 + 2.73 + T_gal_total) 

    noise_power = (T_sys ** 2 * chi(z)**2 * (lambda_z(z) * 4 * np.pi * f_sky)/ ( 2.0 * N_dish * t_obs * 3600))

    return noise_power

In [ ]:
# Defining the fitting model

def calculate_P_fit(k, P_0, m, n, k_T0):
    x = np.log(k)/np.log(k_T0) - 1
    P_fit = np.where(k < k_T0, P_0 ** (1 - m * x**2), P_0 ** (1 - n * x**2))
    return P_fit

In [ ]:
# Plotting the fitting model

P_0 = 3700
m = 0.35
n = 0.45
k_T0 = 0.0167

y1_values = calculate_P_fit(k_values, P_0, m, n, k_T0)
plt.loglog(k_values, y1_values)
plt.show()

In [ ]:
# Defining the N_modes function 

def N_modes(k, z, survey):
    V_survey = survey["V_survey"]
    k_m = survey["k_min"]
    delta_k = 2 * k_m * 1e-3
    N_modes = V_survey * (4 * np.pi * k ** 2 * delta_k) / (2 * np.pi) ** 3
    return N_modes

In [ ]:
# The error function

def Error_PK(k_values, z, survey, theta):
    m, n, P0, kT0 = theta
    V_survey = survey["V_survey"]
    Errors = (calculate_P_fit(k_values, P0, m, n, kT0) + calculate_noise_power(survey, k_values, z)) / np.sqrt(N_modes(k_values, z, survey))
    return Errors

In [ ]:
# Defining the Likelihood, the Priors and the Posterior distributions

def log_likelihood(theta, k_values, z, survey):
    m, n, P0, kT0 = theta
    model_errors = Error_PK(k_values, z, survey, theta)
    P_fit = calculate_P_fit(k_values, P0, m, n, kT0)
    data = P_HI_kz(k_values,z) + calculate_noise_power(survey, k_values, z)
    chi = P_fit - data
    return -0.5 * np.sum((chi/model_errors) ** 2)

def log_prior(theta):
    m, n, P0, kT0 = theta
    if 0 < m < 10 and 0 < n < 10 and 2e2 < P0 < 5e5 and 0.009 < kT0 < 0.35:
        return 0.0
    return -np.inf

def log_probability(theta, k_values, z, survey):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, k_values, z, survey)

In [ ]:
def scale_down_by_1e5(x, pos):
    return f"{x/1e5:.2f}"

def run_mcmc(params):
    i, survey_info = params
    pos = np.array([0.30, 0.85, 2.09e3, 0.015]) + 1e-4 * np.random.randn(35, 4)
    nwalkers, ndim = pos.shape

    with Pool() as pool:
        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(k_values, z_value, survey_info), pool=pool)
        sampler.run_mcmc(pos, 3000, progress=True)

    flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
    samples_list.append(flat_samples)
    gd_samples = MCSamples(samples=flat_samples, names=labels, labels=labels, label=legend_labels[i])
    gd_samples_list.append(gd_samples)

samples_list = []
gd_samples_list = []cond
labels = ['m', 'n', 'P0', 'kT0']
legend_labels = [survey_info["name"] for survey_info in survey_params]

# Run MCMC in parallel
with Pool() as pool:
    pool.map(run_mcmc, enumerate(survey_params))

g = plots.get_subplot_plotter()
g.triangle_plot(gd_samples_list, filled=True)

# Apply formatter to axis
ax_x = g.subplots[-1, 2]
ax_x.xaxis.set_major_formatter(ticker.FuncFormatter(scale_down_by_1e5))
ax_y = g.subplots[2, 1]
if ax_y is not None:
    ax_y.yaxis.set_major_formatter(ticker.FuncFormatter(scale_down_by_1e5))
else:
    print("Subplot for P_0 on the y-axis not found.")

plt.savefig("The_Real_MCMC_Plot.pdf", dpi=300)
plt.savefig('triangle_plot_mcmc.svg', format='svg')
plt.show()